In [ ]:
import os
import subprocess
from functools import partial
from utils import travis_op, appveyor_op, apply_to_all_projects


NEW_COMMAND = (
'''
# If there is a newer build queued for the same PR, cancel this one.
# The AppVeyor 'rollout builds' option is supposed to serve the same
# purpose but it is problematic because it tends to cancel builds pushed
# directly to master instead of just PR builds (or the converse).
# credits: JuliaLang developers.
- ps: if ($env:APPVEYOR_PULL_REQUEST_NUMBER -and $env:APPVEYOR_BUILD_NUMBER -ne ((Invoke-RestMethod `
    https://ci.appveyor.com/api/projects/$env:APPVEYOR_ACCOUNT_NAME/$env:APPVEYOR_PROJECT_SLUG/history?recordsNumber=50).builds | `
    Where-Object pullRequestId -eq $env:APPVEYOR_PULL_REQUEST_NUMBER)[0].buildNumber) { `
      throw "There are newer queued builds for this pull request, failing early." }''')

working_dir = '/Users/jab08/_bump_condaci'
        
def add_new_command_to_string(string):
    if NEW_COMMAND in string:
        print('already has command, skipping')
        return string
    else:
        before, after = string.split('init:')
        new_string = before + 'init:' + NEW_COMMAND + after
        return new_string

appveyor_add_command = partial(appveyor_op, add_new_command_to_string)

def bumps_repo(repo_dir):
    os.chdir(repo_dir)
    did_bump_appveyor = appveyor_add_command(repo_dir)
    if did_bump_appveyor:
        print(subprocess.check_output(['git', 'commit', '-am', 'bail AppVeyor builds if newer PR bulid added']))
        print(subprocess.check_output(['git', 'push', 'origin', 'master']))
    else:
        print('{} is already up to date'.format(repo_dir))

In [ ]:
apply_to_all_projects(working_dir, bump_repo)